# Simple MNIST convnet

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2015/06/19<br>
**Last modified:** 2020/04/21<br>
**Description:** A simple convnet that achieves ~99% test accuracy on MNIST.

In [ ]:
# Ref: https://tf.keras.io/examples/vision/mnist_convnet/

## Setup

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import tensorflow as tf

## Prepare the data

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Build the model

In [4]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape, name="input_0"),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

## Train the model

In [5]:
# batch_size = 128
# epochs = 15

# model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

## Evaluate the trained model

In [6]:
# score = model.evaluate(x_test, y_test, verbose=0)
# print("Test loss:", score[0])
# print("Test accuracy:", score[1])

In [7]:
# iterate over the layers and export models from first to current layer to h5 files
for i in range(0, len(model.layers)):
    model_i = tf.keras.Model(inputs=model.inputs, outputs=model.layers[i].output, name=f"model_{i}")
    model_i.save(f"model_{i}/model.h5")
    # print(model_i.summary())

In [11]:
import tf2onnx

spec = tf.TensorSpec([1, 28, 28, 1], tf.float32, name="input_0")

for i in range(0, len(model.layers)):
    model_i = tf.keras.models.load_model(f'model_{i}/model.h5')
    tf2onnx.convert.from_keras(
        model_i,
        input_signature=[spec],
        inputs_as_nchw=['input_0'],
        opset=12,
        output_path=f'model_{i}/model.onnx'
    )
    tf2onnx.convert.from_keras(
        model_i,
        input_signature=[spec],
        inputs_as_nchw=['input'],
        opset=18,
        output_path=f'model_{i}/opset18.onnx'
    )

In [12]:
# Ref: https://github.com/zkonduit/ezkl/blob/bceac2fab530fd01701aec3d8018ce318f6c42e1/examples/notebooks/mnist_vae.ipynb
!RUST_LOG=trace

# import os
import ezkl
import json


for i in range(0, len(model.layers)):
    model_path = os.path.join(f'model_{i}/model.onnx')
    settings_path = os.path.join(f'model_{i}/settings.json')

    res = ezkl.gen_settings(model_path, settings_path)
    assert res == True

    # read the settings from json
    with open(settings_path, 'r') as f:
        settings = json.load(f)
    
    # print the "num_rows" from the settings
    print(f"Model {i} num_rows: {settings['num_rows']}")


Model 0 num_rows: 184656
Model 1 num_rows: 236032
Model 2 num_rows: 1384793
Model 3 num_rows: 1399322
Model 4 num_rows: 1399322
Model 5 num_rows: 1399324
Model 6 num_rows: 1406581


In [13]:
import sys
sys.path.append('..')
from keras2circom.keras2circom import circom, transpiler
circom.dir_parse('../keras2circom/node_modules/circomlib-ml/circuits/', skips=['util.circom', 'circomlib-matrix', 'circomlib', 'crypto'])

In [14]:
for i in range(0, len(model.layers)):
    args = {
        '<model.h5>': f'model_{i}/model.h5',
        '--output': f'model_{i}',
        '--raw': False,
        '--decimals': "18"
    }
    transpiler.transpile(args['<model.h5>'], args['--output'], args['--raw'], args['--decimals'])

In [11]:
import subprocess

commands = []

for i in range(0, len(model.layers)):
    commands.append(['circom', f'model_{i}/circuit.circom', '--r1cs', '-o', f'model_{i}'])

# Run each command sequentially and print their output
for cmd in commands:
    subprocess.call(cmd)

template instances: 11
non-linear constraints: 11378432
linear constraints: 0
public inputs: 0
private inputs: 66000 (22704 belong to witness)
public outputs: 21632
wires: 11357873
labels: 23537825
Written successfully: model_0/circuit.r1cs
Everything went okay
template instances: 17
non-linear constraints: 16867552
linear constraints: 0
public inputs: 0
private inputs: 71408 (44336 belong to witness)
public outputs: 5408
wires: 16825361
labels: 29362241
Written successfully: model_1/circuit.r1cs
Everything went okay
template instances: 19
non-linear constraints: 23101472
linear constraints: 0
public inputs: 0
private inputs: 113136 (75920 belong to witness)
public outputs: 7744
wires: 23069969
labels: 49347777
Written successfully: model_2/circuit.r1cs
Everything went okay
template instances: 20
non-linear constraints: 24725472
linear constraints: 0
public inputs: 0
private inputs: 114736 (83664 belong to witness)
public outputs: 1600
wires: 24687569
labels: 51070977
Written successfu